# 1. 콜금리 수집
네이버 금융에서 콜글리 데이터 수집

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import html2text

def saveCallRate() :
    df = pd.DataFrame(columns=['date','close','diff','change'])

    # 페이지수 수기로 계산하여 반복문 설정
    for pageIndex in range(1, 652):
        url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(pageIndex)
        page = requests.get(url)

        soup = BeautifulSoup(page.content, 'html.parser')
        trs = soup.find('tbody').find_all('tr')

        for tr in trs :
            tds = tr.find_all("td")
            tmp = [tds[0].text.strip(),float(tds[1].text.strip()),float(tds[2].text.strip()),tds[3].text.strip()]
            df.loc[len(df)] = tmp
            #print(tmp)        
    df.to_csv('./data/market/callrate.csv', index=False)
    
saveCallRate()

# 2. 1개월전 데이터와 비교하여 라벨부여

In [10]:
import pandas as pd


callrate = pd.read_csv('./data/market/callrate.csv')
callrate.sort_values(by='date', ascending=False)

,date,close,diff,change
0,2019.05.21,1.73,0.03,-1.70%
1,2019.05.20,1.76,0.02,+ 1.15%
2,2019.05.17,1.74,0.01,-0.57%
3,2019.05.16,1.75,0.01,-0.57%
4,2019.05.15,1.76,0.03,+ 1.73%
5,2019.05.14,1.73,0.02,-1.14%
6,2019.05.13,1.75,0.01,+ 0.57%
7,2019.05.10,1.74,0.01,+ 0.58%
8,2019.05.09,1.73,0.00,0.00%
9,2019.05.08,1.73,0.00,0.00%


In [16]:
import pandas as pd

def applyCallrateLable() :
    callrate = pd.read_csv('./data/market/callrate.csv')

    callrate['pre_date'] = ''
    callrate['pre_close'] = ''
    callrate['pre_change'] = ''
    callrate['lable'] = 0

    for i, row in callrate.iterrows() :
        if callrate['date'][i] >= '2005.01.01' and callrate['date'][i] <= '2017.12.31' :
            #print(i)
            #print(i+21)
            callrate['pre_date'][i] = callrate['date'][i+21]
            callrate['pre_close'][i] = callrate['close'][i+21]
            callrate['pre_change'][i] = row['close'] - callrate['close'][i+21]
            if callrate['pre_change'][i] > 0 :
                callrate['lable'][i] = 1
            elif callrate['pre_change'][i] < 0 :
                callrate['lable'][i] = -1
            print('{}~{} : {} [{}]'.format(callrate['pre_date'][i],callrate['date'][i],callrate['pre_change'][i],callrate['lable'][i]))

        if callrate['date'][i] == '2005.01.01' :
            break

    callrate.loc[callrate.date >= '2005.01.01'].loc[callrate.date <= '2017.12.31'].to_csv('./data/market/callrate_lable.csv', index=False)

applyCallrateLable() 

/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/leeminho/anaconda3/lib/python3.6/site-pack

2017.11.29~2017.12.29 : 0.32000000000000006 [1]
2017.11.28~2017.12.28 : 0.29000000000000004 [1]
2017.11.27~2017.12.27 : 0.25 [1]
2017.11.24~2017.12.26 : 0.27 [1]
2017.11.23~2017.12.22 : 0.26 [1]
2017.11.22~2017.12.21 : 0.29000000000000004 [1]
2017.11.21~2017.12.20 : 0.26 [1]
2017.11.20~2017.12.19 : 0.25 [1]
2017.11.17~2017.12.18 : 0.26 [1]
2017.11.16~2017.12.15 : 0.27 [1]
2017.11.15~2017.12.14 : 0.26 [1]
2017.11.14~2017.12.13 : 0.26 [1]
2017.11.13~2017.12.12 : 0.26 [1]
2017.11.10~2017.12.11 : 0.28 [1]
2017.11.09~2017.12.08 : 0.28 [1]
2017.11.08~2017.12.07 : 0.26 [1]
2017.11.07~2017.12.06 : 0.22999999999999998 [1]
2017.11.06~2017.12.05 : 0.24 [1]
2017.11.03~2017.12.04 : 0.24 [1]
2017.11.02~2017.12.01 : 0.24 [1]
2017.11.01~2017.11.30 : 0.29000000000000004 [1]
2017.10.31~2017.11.29 : 0.0 [0]
2017.10.30~2017.11.28 : 0.030000000000000027 [1]
2017.10.27~2017.11.27 : 0.020000000000000018 [1]
2017.10.26~2017.11.24 : 0.0 [0]
2017.10.25~2017.11.23 : 0.010000000000000009 [1]
2017.10.24~2017.11.22

/Users/leeminho/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2017.10.11~2017.11.09 : 0.0 [0]
2017.10.10~2017.11.08 : -0.030000000000000027 [-1]
2017.09.29~2017.11.07 : -0.020000000000000018 [-1]
2017.09.28~2017.11.06 : -0.030000000000000027 [-1]
2017.09.27~2017.11.03 : -0.020000000000000018 [-1]
2017.09.26~2017.11.02 : -0.010000000000000009 [-1]
2017.09.25~2017.11.01 : -0.020000000000000018 [-1]
2017.09.22~2017.10.31 : 0.010000000000000009 [1]
2017.09.21~2017.10.30 : -0.020000000000000018 [-1]
2017.09.20~2017.10.27 : -0.010000000000000009 [-1]
2017.09.19~2017.10.26 : 0.010000000000000009 [1]
2017.09.18~2017.10.25 : -0.010000000000000009 [-1]
2017.09.15~2017.10.24 : -0.010000000000000009 [-1]
2017.09.14~2017.10.23 : -0.020000000000000018 [-1]
2017.09.13~2017.10.20 : -0.020000000000000018 [-1]
2017.09.12~2017.10.19 : -0.020000000000000018 [-1]
2017.09.11~2017.10.18 : -0.010000000000000009 [-1]
2017.09.08~2017.10.17 : -0.010000000000000009 [-1]
2017.09.07~2017.10.16 : 0.0 [0]
2017.09.06~2017.10.13 : 0.010000000000000009 [1]
2017.09.05~2017.10.12 : 

2016.12.30~2017.02.01 : -0.040000000000000036 [-1]
2016.12.29~2017.01.31 : 0.010000000000000009 [1]
2016.12.28~2017.01.26 : 0.030000000000000027 [1]
2016.12.27~2017.01.25 : 0.010000000000000009 [1]
2016.12.26~2017.01.24 : 0.010000000000000009 [1]
2016.12.23~2017.01.23 : 0.0 [0]
2016.12.22~2017.01.20 : 0.010000000000000009 [1]
2016.12.21~2017.01.19 : 0.020000000000000018 [1]
2016.12.20~2017.01.18 : 0.030000000000000027 [1]
2016.12.19~2017.01.17 : 0.0 [0]
2016.12.16~2017.01.16 : 0.040000000000000036 [1]
2016.12.15~2017.01.13 : 0.020000000000000018 [1]
2016.12.14~2017.01.12 : 0.010000000000000009 [1]
2016.12.13~2017.01.11 : 0.020000000000000018 [1]
2016.12.12~2017.01.10 : 0.020000000000000018 [1]
2016.12.09~2017.01.09 : 0.010000000000000009 [1]
2016.12.08~2017.01.06 : 0.010000000000000009 [1]
2016.12.07~2017.01.05 : 0.010000000000000009 [1]
2016.12.06~2017.01.04 : 0.010000000000000009 [1]
2016.12.05~2017.01.03 : 0.040000000000000036 [1]
2016.12.02~2017.01.02 : 0.10000000000000009 [1]
2016

...

2008.02.05~2008.03.10 : 0.04999999999999982 [1]
2008.02.04~2008.03.07 : 0.040000000000000036 [1]
2008.02.01~2008.03.06 : 0.08999999999999986 [1]
2008.01.31~2008.03.05 : -0.03000000000000025 [-1]
2008.01.30~2008.03.04 : -0.03000000000000025 [-1]
2008.01.29~2008.03.03 : -0.05999999999999961 [-1]
2008.01.28~2008.02.29 : -0.0699999999999994 [-1]
2008.01.25~2008.02.28 : -0.04999999999999982 [-1]
2008.01.24~2008.02.27 : -0.0699999999999994 [-1]
2008.01.23~2008.02.26 : -0.05999999999999961 [-1]
2008.01.22~2008.02.25 : 0.08000000000000007 [1]
2008.01.21~2008.02.22 : 0.07000000000000028 [1]
2008.01.18~2008.02.21 : 0.04999999999999982 [1]
2008.01.17~2008.02.20 : -0.07999999999999918 [-1]
2008.01.16~2008.02.19 : -0.0699999999999994 [-1]
2008.01.15~2008.02.18 : -0.02999999999999936 [-1]
2008.01.14~2008.02.15 : -0.009999999999999787 [-1]
2008.01.11~2008.02.14 : 0.0 [0]
2008.01.10~2008.02.13 : -0.009999999999999787 [-1]
2008.01.09~2008.02.12 : 0.009999999999999787 [1]
2008.01.08~2008.02.11 : 0.01999

2007.05.18~2007.06.20 : -0.010000000000000675 [-1]
2007.05.17~2007.06.19 : -0.020000000000000462 [-1]
2007.05.16~2007.06.18 : -0.020000000000000462 [-1]
2007.05.15~2007.06.15 : -0.020000000000000462 [-1]
2007.05.14~2007.06.14 : -0.03000000000000025 [-1]
2007.05.11~2007.06.13 : -0.03000000000000025 [-1]
2007.05.10~2007.06.12 : -0.08999999999999986 [-1]
2007.05.09~2007.06.11 : -0.08999999999999986 [-1]
2007.05.08~2007.06.08 : -0.11000000000000032 [-1]
2007.05.07~2007.06.07 : -0.27999999999999936 [-1]
2007.05.04~2007.06.05 : -0.20000000000000018 [-1]
2007.05.03~2007.06.04 : -0.23000000000000043 [-1]
2007.05.02~2007.06.01 : -0.25 [-1]
2007.04.30~2007.05.31 : -0.33000000000000007 [-1]
2007.04.27~2007.05.30 : -0.5300000000000002 [-1]
2007.04.26~2007.05.29 : -0.4900000000000002 [-1]
2007.04.25~2007.05.28 : -0.20000000000000018 [-1]
2007.04.24~2007.05.25 : -0.08999999999999986 [-1]
2007.04.23~2007.05.23 : -0.05999999999999961 [-1]
2007.04.20~2007.05.22 : -0.1900000000000004 [-1]
2007.04.19~200

2006.09.05~2006.10.09 : 0.010000000000000675 [1]
2006.09.04~2006.10.04 : -0.019999999999999574 [-1]
2006.09.01~2006.10.02 : -0.040000000000000036 [-1]
2006.08.31~2006.09.29 : -0.009999999999999787 [-1]
2006.08.30~2006.09.28 : -0.010000000000000675 [-1]
2006.08.29~2006.09.27 : -0.010000000000000675 [-1]
2006.08.28~2006.09.26 : -0.010000000000000675 [-1]
2006.08.25~2006.09.25 : -0.009999999999999787 [-1]
2006.08.24~2006.09.22 : 0.0 [0]
2006.08.23~2006.09.21 : -0.009999999999999787 [-1]
2006.08.22~2006.09.20 : 0.0 [0]
2006.08.21~2006.09.19 : -0.010000000000000675 [-1]
2006.08.18~2006.09.18 : -0.010000000000000675 [-1]
2006.08.17~2006.09.15 : -0.020000000000000462 [-1]
2006.08.16~2006.09.14 : -0.020000000000000462 [-1]
2006.08.14~2006.09.13 : -0.020000000000000462 [-1]
2006.08.11~2006.09.12 : -0.03000000000000025 [-1]
2006.08.10~2006.09.11 : -0.020000000000000462 [-1]
2006.08.09~2006.09.08 : 0.2400000000000002 [1]
2006.08.08~2006.09.07 : 0.21999999999999975 [1]
2006.08.07~2006.09.06 : 0.17

2005.12.16~2006.01.16 : 0.0 [0]
2005.12.15~2006.01.13 : -0.010000000000000231 [-1]
2005.12.14~2006.01.12 : -0.009999999999999787 [-1]
2005.12.13~2006.01.11 : 0.0 [0]
2005.12.12~2006.01.10 : 0.0 [0]
2005.12.09~2006.01.09 : 0.010000000000000231 [1]
2005.12.08~2006.01.06 : -0.040000000000000036 [-1]
2005.12.07~2006.01.05 : 0.22999999999999998 [1]
2005.12.06~2006.01.04 : 0.31000000000000005 [1]
2005.12.05~2006.01.03 : 0.26000000000000023 [1]
2005.12.02~2006.01.02 : 0.25 [1]
2005.12.01~2005.12.30 : 0.2599999999999998 [1]
2005.11.30~2005.12.29 : 0.25 [1]
2005.11.29~2005.12.28 : 0.25 [1]
2005.11.28~2005.12.27 : 0.25 [1]
2005.11.25~2005.12.26 : 0.25 [1]
2005.11.24~2005.12.23 : 0.2400000000000002 [1]
2005.11.23~2005.12.22 : 0.18999999999999995 [1]
2005.11.22~2005.12.21 : 0.2400000000000002 [1]
2005.11.21~2005.12.20 : 0.31000000000000005 [1]
2005.11.18~2005.12.19 : 0.25 [1]
2005.11.17~2005.12.16 : 0.25 [1]
2005.11.16~2005.12.15 : 0.25 [1]
2005.11.15~2005.12.14 : 0.2599999999999998 [1]
2005.11.14

2005.03.28~2005.04.27 : 0.020000000000000018 [1]
2005.03.25~2005.04.26 : 0.0 [0]
2005.03.24~2005.04.25 : 0.029999999999999805 [1]
2005.03.23~2005.04.22 : 0.010000000000000231 [1]
2005.03.22~2005.04.21 : 0.06000000000000005 [1]
2005.03.21~2005.04.20 : 0.09999999999999964 [1]
2005.03.18~2005.04.19 : 0.029999999999999805 [1]
2005.03.17~2005.04.18 : 0.020000000000000018 [1]
2005.03.16~2005.04.15 : 0.029999999999999805 [1]
2005.03.15~2005.04.14 : 0.009999999999999787 [1]
2005.03.14~2005.04.13 : 0.009999999999999787 [1]
2005.03.11~2005.04.12 : -0.009999999999999787 [-1]
2005.03.10~2005.04.11 : -0.009999999999999787 [-1]
2005.03.09~2005.04.08 : -0.010000000000000231 [-1]
2005.03.08~2005.04.07 : -0.040000000000000036 [-1]
2005.03.07~2005.04.06 : 0.06000000000000005 [1]
2005.03.04~2005.04.04 : -0.009999999999999787 [-1]
2005.03.03~2005.04.01 : -0.020000000000000018 [-1]
2005.03.02~2005.03.31 : -0.020000000000000018 [-1]
2005.02.28~2005.03.30 : -0.029999999999999805 [-1]
2005.02.25~2005.03.29 : 

In [17]:
import pandas as pd

pd.read_csv('./data/market/callrate_lable.csv')

,date,close,diff,change,pre_date,pre_close,pre_change,lable
0,2017.12.29,1.58,0.03,+ 1.94%,2017.11.29,1.26,0.32,1
1,2017.12.28,1.55,0.04,+ 2.65%,2017.11.28,1.26,0.29,1
2,2017.12.27,1.51,0.01,-0.66%,2017.11.27,1.26,0.25,1
3,2017.12.26,1.52,0.01,+ 0.66%,2017.11.24,1.25,0.27,1
4,2017.12.22,1.51,0.02,-1.31%,2017.11.23,1.25,0.26,1
5,2017.12.21,1.53,0.02,+ 1.32%,2017.11.22,1.24,0.29,1
6,2017.12.20,1.51,0.01,+ 0.67%,2017.11.21,1.25,0.26,1
7,2017.12.19,1.50,0.00,0.00%,2017.11.20,1.25,0.25,1
8,2017.12.18,1.50,0.01,-0.66%,2017.11.17,1.24,0.26,1
9,2017.12.15,1.51,0.01,+ 0.67%,2017.11.16,1.24,0.27,1
